# Design patterns with first-class function

Although design patterns are language-independent, that does not mean every pattrn applies to every language.

In particular, in the context of languages with first-class functions, Norvig suggests rethinking the Strategy, Command, Template Method and Visitor patterns. The general idea is : you can replace instances of some participant class in these patterns with simple functions, reducing a lot of boilerplate code.

refactor Strategyusing function objects, and discuss a similar approach to simplifying the Command pattern.

## 1. Case Study : refactoring Strategy

Strategy is a good example of a design pattern that can be simpler in Python if you leverage functions as first class objects. 

Define a family of algorithms, encapsulate each one, and make them interchangeable. Strategy lets the algorithm vary indenpendently from clients that use it.


- Contexnt: Provides a sevice by delegating some computation to interchangeable components that implement alternative algorithms. In  the e-commerce example, the context is an Order, which is configured to apply a promotional discount according to one of several algorithms.

- Strategy : The interface common to the components that implement the different algorithms. In our example, this role is played by an abstract class called Promotion.

- Concrete Strategy : One of the concrete subclass of Strategy.




## Classic Strategy

In [1]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price =  price
        
    def total(self):
        return self.price * self.quantity
    
class Order : # The Context
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    def due(self):
        if self.promotion is None:
            discount = 0
            
        else:
            discount = self.promotion.discount(self)
            return self.total() - discount
        
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())
    
class Promotion(ABC): # the Strategy : an Abstract Base Class
    
    @abstractmethod
    def discount(self, order):
        """Return discount as a positive dollar amount"""
        
class FidelityPromo(Promotion): # first Concrente Strategy
    """5% discount for customers with 1000 or more fidelity points"""
    
    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >=1000 else 0
    
class BulkItemPromo(Promotion): # second Concrete Strategy
    """10% discount for each LineItem with 20 or more units"""
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1
            return discount
        
class LargeOrderPromo(Promotion): #third Concrete Strategy
    """7% discount for orders with 10 or more distinct items"""
    
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total()*0.07
        return 0

In [2]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, 0.5),
       LineItem('apple', 10, 1.5),
       LineItem('watermellon', 5, 5.0)]


In [3]:
# The FideliryPromo promotion gives no discount to joe
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [4]:
# ann gets a 5% discount because she has at least 1000 points
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [5]:
banana_cart = [LineItem('banana', 30, 0.5),
              LineItem('apple', 10, 1.5)]

In [6]:
# Thanks to the BulkItemPromo, joe gets a $1.50 discount on the bananas.
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [7]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

In [8]:
print(type(long_order))

<class 'list'>


In [10]:
a = list(long_order)
print(type(a))

<class 'list'>


In [12]:
print(a)
print(long_order)

[<__main__.LineItem object at 0x000001EBBDAC8710>, <__main__.LineItem object at 0x000001EBBDAC8B70>, <__main__.LineItem object at 0x000001EBBDAC8D30>, <__main__.LineItem object at 0x000001EBBDAC8AC8>, <__main__.LineItem object at 0x000001EBBDAC8E80>, <__main__.LineItem object at 0x000001EBBDAC8E48>, <__main__.LineItem object at 0x000001EBBDAC8780>, <__main__.LineItem object at 0x000001EBBDAC8DA0>, <__main__.LineItem object at 0x000001EBBDAC8E10>, <__main__.LineItem object at 0x000001EBBDAC8EF0>]
[<__main__.LineItem object at 0x000001EBBDAC8710>, <__main__.LineItem object at 0x000001EBBDAC8B70>, <__main__.LineItem object at 0x000001EBBDAC8D30>, <__main__.LineItem object at 0x000001EBBDAC8AC8>, <__main__.LineItem object at 0x000001EBBDAC8E80>, <__main__.LineItem object at 0x000001EBBDAC8E48>, <__main__.LineItem object at 0x000001EBBDAC8780>, <__main__.LineItem object at 0x000001EBBDAC8DA0>, <__main__.LineItem object at 0x000001EBBDAC8E10>, <__main__.LineItem object at 0x000001EBBDAC8EF0>

In [13]:
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [14]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

## Function-oriented Strategy

Same funcionality can implemented with less code.

Each concrete strategy is a class with a single method, discount.

Furthermore, the strategy instances have no state(no instance attributes).

Replacing the concrete strategies with simple funcions, and removing the Promo abstract class.

In [1]:
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity

class Order: #the Context
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount=0
            
        else:
            discount = self.promotion(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}'
        return fmt.format(self.total(), self.due())
    
    
def fidelity_promo(order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

def large_order_promo(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

In [2]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, 0.5),
       LineItem('apple', 10, 1.5),
       LineItem('watermellon', 5, 5.0)]

In [3]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00

In [4]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90

In [5]:
banana_cart=[LineItem('banana', 30, 0.5),
            LineItem('apple', 10, 1.5)]

In [6]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50

In [7]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

In [8]:
Order(joe, long_order, large_order_promo)

<Order total: 10.00 due: 9.30

In [9]:
Order(joe, cart, large_order_promo)

<Order total: 42.00 due: 42.00

There is no need to instantiate a new promotion object with each new order: the functions are ready to use.

Strategy objects often make good flyweights.

A flyweight is a shared object that can be used in multiple context simulatenously.

Often concrete strategies have no internal state, they only deal with data from the context. If that is the case, then by all means use plain old functions instead of coding single-method claases

- function is more lightweight than an instance of a user-defiend class

- no need for Flyweight

- is a shared object that can be used in multiple contexts.

## Choosing the best strategy : simple approach

create a meta-strategy that selects the best available discount for a given order.

In [40]:
# list of the strategies implemented as functions
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    # using a generator expression, apply each of the functions from proms to the order.
    return max(promo(order) for promo in promos)


In [41]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30

In [42]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50

In [43]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90

## Finding strategies in a module

there is some duplication that could lead to a subtle bug : to add a new promotion strategy we need to code the function and remeber to add it to the promos list.

- globals(): Return a dictionary representing the current global symbol table.

In [10]:
globals()

{'Customer': __main__.Customer,
 'In': ['',
  "from collections import namedtuple\n\nCustomer = namedtuple('Customer', 'name fidelity')\n\nclass LineItem:\n    \n    def __init__(self, product, quantity, price):\n        self.product = product\n        self.quantity = quantity\n        self.price = price\n    \n    def total(self):\n        return self.price * self.quantity\n\nclass Order: #the Context\n    \n    def __init__(self, customer, cart, promotion=None):\n        self.customer = customer\n        self.cart = list(cart)\n        self.promotion = promotion\n        \n    def total(self):\n        if not hasattr(self, '__total'):\n            self.__total = sum(item.total() for item in self.cart)\n        return self.__total\n    \n    def due(self):\n        if self.promotion is None:\n            discount=0\n            \n        else:\n            discount = self.promotion(self)\n        return self.total() - discount\n    \n    def __repr__(self):\n        fmt = '<Order tota

In [11]:
globals()['bulk_item_promo']

<function __main__.bulk_item_promo(order)>

In [13]:
# Select only names that end with the _promo suffix, Filter out best_promo itself,
# to avoid an infinit recursion.
promos = [globals()[name] for name in globals() if name.endswith('_promo') and name!='best_promo']

def best_promo(order):
    return max(promo(order) for promo in promos)

Another way of collection the available promotions would be to create a module and put all the strategy functions there, except for best_promo

list of strategy functions is built by introspection of a separate module called `promotions`

In [14]:
import inspect

promos = [func for name, func in inspect.getmembers(promotions, inspect.isfunction)]

def best_promo(order):
    return max(promo(order) for promo in promos)

NameError: name 'promotions' is not defined

function inspect.getmembers returns the attributes of an object(the promotions modules).

We use inspect.insfunction to get only the functions from the module.

it works regardless of the name given to the functions; all the matters is that the promotions module contains only functions that calculate discounts given orders.


## 2. Command

Each command have a different receiver: the object that implements the action. For PasteCommand, the receiver is the Docuent, For OpenCommand the receiver is the application.

The goal of Command is to decouple an object that invokes an operation(the Invoker) from the provider object that implements it(the Receiver)

The idea is to put a Command object between the two, implementing an interfcae with a single method, `execute`, which calles some method in the Receiver to perform the desired operation.

That way the Invoker does not need to know the interface of the Receiver, and different receivers can be adapted through different Command subclasses.

The Invoker is configured with a concrete command and calls its `execute` method to operate it.

Instead of giving the Invoker a Command instance, we can simply give it a function. 

Invoker can just call command().

The MacroCommand can be implemented with a class implementing `__call__`

In [16]:
class MacroCommand:
    
    # Building a list from the commands arguments
    # ensures that it is iterable and keeps a local copy of the command references
    # in each MacroCommand instance.
    def __init__(self, commands):
        # list() gives a new list.
        self.commands = list(commands)
        
    def __call__(self):
        for command in self.commands:
            command()

In [15]:
a = [1,2,3]

print(a)

print(list(a))

[1, 2, 3]
[1, 2, 3]


## 3. Chapter summary

As Peter Norvig pointed out "16 of 23 patterns have qualitatively simpler implementation in Lisp or Dylan than in C++ for at least some uses of each pattern."

functions or callable objects provide a more natural way of implementing callbacks in Python than mimicking the Strategy or the Command patterns.
